<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Data_collection_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import of the training data

from google.colab import files
uploaded = files.upload()

# Import of moduls

!pip install wikidata --quiet
!pip install datasets --quiet
!pip install spacy --quiet
!python -m spacy download en_core_web_md

import matplotlib.pyplot as plt
import pandas as pd
from wikidata.client import Client
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
import spacy
import seaborn as sns
from collections import Counter

from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading the data
train_df = pd.read_csv('[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep='\t')

Saving [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv to [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 46.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Data collection : en_core_web_md

In [2]:
# Loading the pretrained model
nlp = spacy.load("en_core_web_md")

In [3]:
def extract_linguistic_features(text):
    doc = nlp(text)

    # Extract the number of nuns, verbs and adjectives in the text
    pos_counts = Counter([token.pos_ for token in doc])
    num_nouns = pos_counts.get("NOUN", 0)
    num_verbs = pos_counts.get("VERB", 0)
    num_adjectives = pos_counts.get("ADJ", 0) # adjectives can indicate a subjective tone

    # NER: named entities recognition
    ent_labels = [ent.label_ for ent in doc.ents]
    has_location = int("GPE" in ent_labels or "LOC" in ent_labels) # GPE = Geo political entity
    has_ethnic_group = int("NORP" in ent_labels)  # NORP = Nationalities, religious or political groups
    has_event = int("EVENT" in ent_labels)

    return {
        "descr_num_nouns": num_nouns,
        "descr_num_verbs": num_verbs,
        "descr_num_adjectives": num_adjectives,
        "descr_has_location": has_location,
        "descr_has_ethnic_group": has_ethnic_group,
        "descr_has_event": has_event,
    }

In [5]:
# Extract the features
df_features = train_df['description'].apply(extract_linguistic_features).apply(pd.Series)

# Add it to the dataframe
train_df = pd.concat([train_df, df_features], axis=1)

# Visualisation
train_df

,item,name,description,type,category,subcategory,label,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,descr_has_ethnic_group,descr_has_event
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,2,0,1,0,1,0
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,4,0,1,0,1,0
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,1,0,1,0,1,0
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,2,0,1,0,1,0
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,1,0,0,1,0,0
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,3,0,3,1,0,0
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,2,0,2,1,0,0
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,2,0,1,1,0,0


In [6]:
# Exportation

train_df.to_json("train_df_descr_analyse.json", orient="records", lines=True)
files.download('train_df_descr_analyse.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>